# Introduction

### Questions

1. Do I come up with a way to map equivalent expressions?
4. single eqn, multi eqn, curriculum, train/test split
6. Robust to trig/log/exp/sqrt? Pare down other actions
7. GCN... dummy! Get this working pronto.
8 Embedding
8. Visualizer
9. i/e rewards
10. Profile
- Check if later equations are solvable IN THEORY, with new, smaller action space

10. Alternate evaluators: dont run greedy during evaluation. Let run multiple times, and check Tsolve .... we're inventing.

In [87]:
from sympy import symbols
from utils.utils_env import *
from utils.custom_functions import *

a,b,c,d,e,x = symbols('a b c d e x')

lhs, rhs = (a*x+b)/(c*x+d) + e, 0
get_ordered_sub_expressions(lhs)

actions = [ (truediv,1/(c*x + d)), (custom_ratsimp, None), (custom_collect,x), (sub,d*e), (sub,b), (truediv, a + c*e)]
for (op,term) in actions:
    lhs, rhs = op(lhs, term), op(rhs, term)
    print(f'{lhs} = {rhs}')

(e + (a*x + b)/(c*x + d))*(c*x + d) = 0
a*x + b + c*e*x + d*e = 0
b + d*e + x*(a + c*e) = 0
b + x*(a + c*e) = -d*e
x*(a + c*e) = -b - d*e
x = (-b - d*e)/(a + c*e)


In [82]:
import time
from collections import deque
from sympy import symbols
from utils.utils_env import *
from utils.custom_functions import *

def get_ordered_sub_expressions_old(expr):
    if expr == 0:
        return []
    
    sub_expressions = set()
    queue = [expr]  # BFS queue
    
    while queue:
        e = queue.pop(0)
        
        if e not in sub_expressions and 1/e not in sub_expressions and e not in [-1, 0, 1]:
            sub_expressions.add(e)
            if not e.is_Atom:
                queue.extend(e.args)
    
    sub_expressions.discard(expr)
    return sorted(sub_expressions, key=lambda x: (len(str(x)), str(x)))

def get_ordered_sub_expressions_new(expr):
    if expr == 0:
        return []
    
    sub_expressions = set()
    queue = deque([expr])
    
    while queue:
        e = queue.popleft()
        
        if e in sub_expressions or 1/e in sub_expressions or e in [-1, 0, 1]:
            continue
        
        sub_expressions.add(e)
        if not e.is_Atom:
            queue.extend(e.args)
    
    sub_expressions.discard(expr)
    return sorted(sub_expressions, key=lambda x: (x.count_ops(), x))

    #return sorted(sub_expressions, key=str)

def benchmark_function(func, expr, num_runs=100):
    start_time = time.time()
    for _ in range(num_runs):
        func(expr)
    return (time.time() - start_time) / num_runs  # Average time per run

a, b, c, d, e, x = symbols('a b c d e x')

test_expressions = [
    a*x + b,
    (a*x + b) * (c*x + d),
    (a*x + b)/(c*x + d) + e,
    a*b + c*d + e*x + x**2
]

for expr in test_expressions:
    time_old = benchmark_function(get_ordered_sub_expressions_old, expr)
    time_new = benchmark_function(get_ordered_sub_expressions_new, expr)
    
    output_old = get_ordered_sub_expressions_old(expr)
    output_new = get_ordered_sub_expressions_new(expr)
    same_output = output_old == output_new
    
    print(f"Expression: {expr}")
    print(f"Old Version Time: {time_old:.6f} sec")
    print(f"New Version Time: {time_new:.6f} sec")
    print(f"Old output: {output_old}")
    print(f"new output: {output_new}")
    print(f"Speedup: {time_old / time_new:.2f}x")
    print(f"Outputs Match: {same_output}")
    print("-" * 50)

TypeError: cannot determine truth value of Relational

In [83]:
import torch
print("PyTorch Version:", torch.__version__)
print("MPS Available:", torch.backends.mps.is_available())
print("MPS Built:", torch.backends.mps.is_built())

PyTorch Version: 2.3.1
MPS Available: True
MPS Built: True
